<a href="https://colab.research.google.com/github/pikanaeri/plm-model-comparison/blob/main/efam-performance/EFAM_Embedding_ProstT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Starting Procedure (Computer Setup)

In [ ]:
sudo apt-get update -qq
sudo apt-get install -y tmux
tmux

In [ ]:
sudo apt-get update
sudo apt-get install bzip2 libxml2-dev
sudo apt upgrade
sudo apt-get install wget
wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
bash Anaconda3-2022.05-Linux-x86_64.sh
rm Anaconda3-2022.05-Linux-x86_64.sh

In [ ]:
sudo fallocate -l 15G /swapfile
sudo chmod 600 /swapfile
ls -lh /swapfile
sudo mkswap /swapfile
sudo swapon /swapfile

#@markdown Append "/swapfile swap swap defaults 0 0" to the /etc/fstab file using a text editor of your choice
sudo nano /etc/fstab

/swapfile swap swap defaults 0 0

In [ ]:
sudo apt-get install git
sudo apt install python3-pip

python3 --version
pip3 --version

mkdir final_embeddings_uppercase

SyntaxError: invalid syntax (<ipython-input-1-3b0e3c477eff>, line 1)

In [ ]:
pip install transformers
pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


## Embedding using ProtT5




In [ ]:
touch script.py
sudo nano script.py
#paste code below into the editor, then ctrl+o, enter and ctrl+x to save
python script.py

In [ ]:
# In[1]:


import pandas as pd


# In[12]:


x = pd.read_csv('efam_phrog_e10.csv', index_col=0)
y = pd.read_csv('PHROG_index_revised_v4_10292022.csv', index_col=0)


# In[18]:


d = dict(zip(list(y.index), y['revised_category_v4']))


# In[22]:


x['label'] = [d[p] for p in x['phrog#']]


# In[27]:


xx = x[x['label'] != 'unknown function']


# In[33]:


len(set(xx['cluster']))

print(len(set(xx['cluster'])))
print(xx)

66196
              cluster       phrog#   Prob       E-value       P-value  Score  \
0        cluster66870  phrog_18345  100.0  2.900000e-50  7.400000e-55  432.9   
1        cluster66870   phrog_9942  100.0  3.200000e-43  8.200000e-48  368.2   
2        cluster66870  phrog_19195  100.0  9.700000e-43  2.500000e-47  343.5   
3        cluster66870  phrog_22896   99.6  2.900000e-19  7.500000e-24  185.6   
4        cluster66870   phrog_6921   99.0  1.000000e-13  2.600000e-18  150.0   
...               ...          ...    ...           ...           ...    ...   
175868  cluster114511   phrog_4758  100.0  4.600000e-38  1.200000e-42  292.1   
175869  cluster114511   phrog_5573   99.6  4.800000e-20  1.200000e-24  172.7   
175870  cluster114511  phrog_30251   99.0  8.200000e-14  2.100000e-18  140.7   
175871  cluster165827  phrog_21992   98.6  3.800000e-11  9.900000e-16   94.6   
175872   cluster19349    phrog_112   99.7  1.600000e-20  4.200000e-25  152.5   

                                 

In [ ]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
import time
import gc
import numpy as np
import pickle as pkl

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))
transformer_link = "Rostlab/ProstT5"
print("Loading: {}".format(transformer_link))
model = T5EncoderModel.from_pretrained(transformer_link)
model.full() if device=='cpu' else model.half()
model = model.to(device)
model = model.eval()

tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False )


#1. Get sequences here
def get_seqs(File_Name):
  names = []
  seqs = [] #should match up with the efam_phrog_e10 file
  reader = open(File_Name, "r")
  name = reader.readline().strip()
  while True: #The sequences are broken up by new lines!!!
    if name == '':
      break
    sequence = ""
    read = reader.readline().strip()
    while read != '' and read[0] != '>':
      sequence += read
      read = reader.readline().strip()
    names.append(name[1:])
    seqs.append(sequence)
    name = read
  reader.close()
  return names, seqs


names, seqs = get_seqs("dereplicated_filtered_proteins_efam_downloaded_10162022.faa")


#2. Embed Families here

def embed_family(cluster_name, max_sz=-1):
  embeddings = []
  nms = []
  seq_nms = x[x['cluster']==cluster_name].index.tolist()
  for i in seq_nms:
    seqs[i] = seqs[i].upper()
    print(seqs[i])
    sequence_examples = [seqs[i]]
    if max_sz == -1 or len(seqs[i]) <= max_sz:
      nms.append(names[i])
      sequence_examples = [" ".join(list(re.sub("[UZOB]", "X", sequence))) for sequence in sequence_examples]
      ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")
      input_ids = torch.tensor(ids['input_ids']).to(device)
      attention_mask = torch.tensor(ids['attention_mask']).to(device)
      with torch.no_grad():
          embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)
      gc.collect()
      torch.cuda.empty_cache()
      for id in range(1):
        emb = embedding_repr.last_hidden_state[id, :len(sequence_examples[id])]
        emb = emb.mean(dim=0)
        emb = emb.cpu().numpy()
        gc.collect()
        torch.cuda.empty_cache()
        embeddings.append(np.array(emb))
  print(seq_nms)
  print(embeddings)
  pkl.dump(nms, open(f"final_embeddings_uppercase/{cluster_name}.txt", "wb"))
  pkl.dump(embeddings, open(f"final_embeddings_uppercase/{cluster_name}.pkl", "wb"))
  return nms, embeddings

#Navigate PHROGs directory here
def nav_dir(sz_cap):
  dct = {}
  clusters = sorted(set(xx['cluster']))
  print(clusters)
  for c in clusters:
    print("embedding ", c)
    s, e = embed_family(c, max_sz=sz_cap)
    for i in range(len(s)):
      dct[s[i]] = e[i]
  f = open("identifier_to_vector_protbert_bdf_prostt5_08132024.pkl", "wb")
  pkl.dump(dct, f)
  f.close()

print("beginning procedure")
nav_dir(sz_cap=5096)


Using device: cuda:0
Loading: Rostlab/prot_t5_xl_uniref50
beginning procedure
['cluster10000', 'cluster100000', 'cluster100003', 'cluster100005', 'cluster100007', 'cluster100008', 'cluster100010', 'cluster100012', 'cluster100013', 'cluster100014', 'cluster100015', 'cluster100016', 'cluster100017', 'cluster100018', 'cluster100019', 'cluster100020', 'cluster100024', 'cluster100026', 'cluster100027', 'cluster100029', 'cluster10003', 'cluster100036', 'cluster100038', 'cluster100039', 'cluster10004', 'cluster100041', 'cluster100045', 'cluster100046', 'cluster100049', 'cluster100050', 'cluster100051', 'cluster100053', 'cluster100055', 'cluster100057', 'cluster100058', 'cluster100059', 'cluster10006', 'cluster100060', 'cluster100063', 'cluster100065', 'cluster100066', 'cluster100068', 'cluster100070', 'cluster100071', 'cluster100072', 'cluster100073', 'cluster100074', 'cluster100078', 'cluster10008', 'cluster100083', 'cluster100084', 'cluster100085', 'cluster100088', 'cluster100092', 'cluster